In [1]:
# %load model.py
import os
from pathlib import Path
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

lines = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images_0 = []
images_non0 = []
measurements_0 = []
measurements_non0 = []
for line in lines[1:]:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './IMG/' + filename
    temp_image = cv2.imread(current_path)
    temp_measurment = float(line[3])
    if temp_measurment == float(0):
        images_0.append(temp_image)
        measurements_0.append(temp_measurment)
        #Duplication of image/steering data with dulpication
        images_0.append(cv2.flip(temp_image,1))
        measurements_0.append(-1*temp_measurment)
    else:
        images_non0.append(temp_image)
        measurements_non0.append(temp_measurment)
        #Duplication of image/steering data with dulpication
        images_non0.append(cv2.flip(temp_image,1))
        measurements_non0.append(-1*temp_measurment)

images = []
measurements = []

for i in range(6):
    measurements.extend(measurements_non0)
    images.extend(images_non0)
del measurements_non0, images_non0

images.extend(images_0)
measurements.extend(measurements_0)
del measurements_0, images_0

images = shuffle(images)
measurements = shuffle(measurements)

count = np.unique(measurements,return_counts=True)
plt.hist(measurements,bins=32)
plt.show()
print(count)

X_train = images[:int(len(images)*0.8)]
y_train = measurements[:int(len(images)*0.8)]
X_validation = images[int(len(images)*0.8+1):]
y_validation = measurements[int(len(images)*0.8+1):]

for i,l in enumerate(y_train):
    path_name = "train/" + str(l)
    file_name = path_name + "/" + str(i) + ".jpg"
    my_dir = Path(path_name)
    if my_dir.is_dir():
        pass
    else:
        os.mkdir(path_name)
    cv2.imwrite(file_name, X_train[i])

for i,l in enumerate(y_validation):
    path_name = "validation/" + str(l)
    file_name = path_name + "/" + str(i) + ".jpg"
    my_dir = Path(path_name)
    if my_dir.is_dir():
        pass
    else:
        os.mkdir(path_name)
    cv2.imwrite(file_name, X_validation[i])
    
print("preprocessing was done")

(array([-1.       , -0.8500001, -0.8000001, -0.       ,  0.8000001,
        0.8500001,  1.       ]), array([ 210,  648,  342, 2396,  342,  648,  210]))
preprocessing was done


In [ ]:
from keras import optimizers
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Flatten, Dense, Lambda, core, Cropping2D, MaxPooling2D
from keras.layers import Convolution2D, normalization, Dropout
from keras.layers.pooling import MaxPooling2D
import keras.backend as K

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
    visible_device_list="0", # specify GPU number
    allow_growth=True
    )
)


def maxout30(input_tensor):
    output_size=30
    shape = input_tensor.get_shape().as_list()
    if shape[1] % output_size == 0:
        return tf.transpose(tf.reduce_max(tf.split(input_tensor,output_size,1),axis=2))
    else:
        raise ValueError("Output size or input tensor size is not fine. Please check it. Reminder need be zero.")

# def maxout10(input_tensor):
#     output_size=10
#     shape = input_tensor.get_shape().as_list()
#     if shape[1] % output_size == 0:
#         return tf.transpose(tf.reduce_max(tf.split(input_tensor,output_size,1),axis=2))
#     else:
#         raise ValueError("Output size or input tensor size is not fine. Please check it. Reminder need be zero.")

epochs = 5
learning_rat = 0.001
batch_size = 32

#Define data generator
train_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
)

validation_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
)

train_generator = train_datagen.flow_from_directory(
    'train',
    batch_size=batch_size,
    target_size = (160,320),
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    'validation',
    batch_size=batch_size,
    target_size = (160,320),
    class_mode='binary'
)


'''
#Define base data
#Data transfer is refer from https://keras.io/ja/models/model/
base_model = InceptionV3(weights='imagenet',include_top=True, input_shape=(160, 320, 3))
for i in range(251):
    base_model.layers.pop()
    
#Data transfer is refer from https://keras.io/ja/models/model/
for layer in base_model.layers:
    layer.trainable = False

new_layer_maxpool1 = MaxPooling2D(pool_size=4,name='New_MaxPooling2D')
x = new_layer_maxpool(base_model.layers[-1].output)
new_convolution = Convolution2D(6,5,5,name='New_Conv2D')
x = new_convolution(x)
# new_layer_flatten = Flatten(name='flatten')
# x = new_layer_flatten(x)
# new_Dense200 = Dense(200,name='Dense200')
# x = new_Dense200(x)
# new_maxout50 = Lambda(maxout50,name='Maxout50')
# x = new_maxout50(x)
# x = Dropout(0.8,name='Drop')(x)
# new_Dense40 = Dense(40,name='Dense40')
# x = new_Dense40(x) 
# new_maxout10 = Lambda(maxout10,name='Maxout10')
# x = new_maxout10(x)
# x = Dropout(0.8,name='Drop2')(x)
out = Dense(1,name='my_dense')(x)

'''

model2 = Sequential()
model2.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model2.add(Convolution2D(6,1,1))
model2.add(Convolution2D(6,5,5,activation="relu"))
model2.add(MaxPooling2D())
model2.add(Convolution2D(6,5,5,activation="relu"))
model2.add(MaxPooling2D())
model2.add(Flatten())
model2.add(Dense(120))
model2.add(Lambda(maxout30))
# model.add(Dense(84))
model2.add(Dense(1))


# model2 = Model(inputs=base_model.input, outputs = out)
model2.summary()

adam = optimizers.Adam(lr=learning_rat, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model2.compile(loss='mse', optimizer=adam)

model_history = model2.fit_generator(
    train_generator,
    steps_per_epoch=500,
    epochs=4,
    validation_data=validation_generator,
    validation_steps=100,
#     samples_per_epoch=10,
#     epoch=4,
)

# model2.fit(X_train, y_train, validation_split=0.2, shuffle=True,nb_epoch=10)

model2.save('model.h5')

Found 17962 images belonging to 8 classes.
Found 4139 images belonging to 8 classes.


/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (1, 1))`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:102: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:104: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 160, 320, 6)       24        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 156, 316, 6)       906       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 78, 158, 6)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 74, 154, 6)        906       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 37, 77, 6)         0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 17094)             0         
__________

In [7]:
# print(dir(model2.layers[-2]))
# print(model2.layers[-2].output_shape)

from keras.applications.resnet50 import preprocess_input, decode_predictions

model3 = Model(inputs=model2.input, outputs=model2.get_layer(index=1).output)
img_path = './IMG/left_2018_01_06_17_22_03_493.jpg'
img = image.load_img(img_path, target_size=(160, 320))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
biz_output = model3.predict(x)

In [8]:
print(biz_output.shape)

print(biz_output[0][0:10])

model2.summary()

# plt.imshow(biz_output[0])

(1, 153600)
[-103.93900299 -110.77899933 -123.68000031 -103.93900299 -111.77899933
 -121.68000031 -103.93900299 -111.77899933 -121.68000031 -103.93900299]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 153600)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 153601    
Total params: 153,601
Trainable params: 153,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(model_history.history.keys())

plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [ ]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras import backend as K


base_model = InceptionV3(weights='imagenet')

for i,l in enumerate(base_model.layers):
    print('index:',i,':name:',l.name,':input_shape:',l.input_shape,':output_shape:',l.output_shape)

model2 = Model(inputs=base_model.input, outputs=base_model.get_layer(index=63).output)

img_path = './IMG/left_2018_01_06_17_22_03_493.jpg'
img = image.load_img(img_path, target_size=( 224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
biz_output = model2.predict(x)

In [ ]:
import matplotlib.pyplot as plt

print(biz_output.shape)
fig = plt.figure(figsize=(20, 40))
for i in range(200):
    exec('ax' + str(i+1) +'= fig.add_subplot(20,10,' + str(i+1) +')') 
    exec('ax' + str(i+1) + ".imshow(biz_output[0,:,:," + str(i) + "], cmap='gray')")
    exec('ax' + str(i+1) + '.tick_params(labelbottom="off",bottom="off")')
    exec('ax' + str(i+1) + '.tick_params(labelleft="off",bottom="off")')
#     exec('ax' + str(i+1) +'.imshow(X_test[' + str(temp_index) + '])')
plt.show()

In [ ]:
viz_output = model.predict(X_train[0:6])
print(model.layers[1].name)

print(X_train[0].shape)
for i,l in enumerate(model.layers):
    print(i,l.name)

model2 = Model(inputs=model.input, outputs=model.get_layer(index=2).output)
viz_output = model2.predict(np.expand_dims(X_train[0],axis=0))
print(viz_output.shape)
print(viz_output)
print(viz_output[0][:][:][0])
# print(block4_pool_features[0][0][0][0].shape)

fig = plt.figure(figsize=(20, 30))
for i in range(4):
    exec('ax' + str(i+1) +'= fig.add_subplot(8,4,' + str(i+1) +')') 
    exec('ax' + str(i+1) + ".imshow(viz_output[0,:,:," + str(i) + "], cmap='gray')")
    exec('ax' + str(i+1) + '.tick_params(labelbottom="off",bottom="off")')
    exec('ax' + str(i+1) + '.tick_params(labelleft="off",bottom="off")')
#     exec('ax' + str(i+1) +'.imshow(X_test[' + str(temp_index) + '])')
plt.show()

In [36]:
from keras.models import Sequential
from keras.layers import Input,Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dropout, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
in_img = Input(shape=(3, 32, 32))

def gen_model():
    in_img = Input(shape=(3, 32, 32))
    x = Convolution2D(12, 3, 3, subsample=(2, 2), border_mode='valid', name='conv1')(in_img)
    x = Activation('relu', name='relu_conv1')(x)
    x = Convolution2D(3, 1, 1, border_mode='valid', name='conv2')(x)
    x = Activation('relu', name='relu_conv2')(x)
    x = GlobalAveragePooling2D()(x)
    o = Activation('softmax', name='loss')(x)
    model = Model(input=in_img, output=[o])
    return model

#parent model
model=gen_model()
model.compile(loss="categorical_crossentropy", optimizer="adam")
model.summary()

#saving model weights
model.save('model_weights.h5')

/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (3, 3), name="conv1", padding="valid", strides=(2, 2))`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (1, 1), name="conv2", padding="valid")`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 3, 32, 32)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 1, 15, 12)         3468      
_________________________________________________________________
relu_conv1 (Activation)      (None, 1, 15, 12)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 1, 15, 3)          39        
_________________________________________________________________
relu_conv2 (Activation)      (None, 1, 15, 3)          0         
_________________________________________________________________
global_average_pooling2d_7 ( (None, 3)                 0         
_________________________________________________________________
loss (Activation)            (None, 3)                 0         
Total para

In [37]:
#loading weights to second model
model2=gen_model()
model2.compile(loss="categorical_crossentropy", optimizer="adam")
model2.load_weights('model_weights.h5', by_name=True)

model2.layers.pop()
model2.layers.pop()
model2.summary()

/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (3, 3), name="conv1", padding="valid", strides=(2, 2))`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, (1, 1), name="conv2", padding="valid")`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 3, 32, 32)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 1, 15, 12)         3468      
_________________________________________________________________
relu_conv1 (Activation)      (None, 1, 15, 12)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 1, 15, 3)          39        
_________________________________________________________________
relu_conv2 (Activation)      (None, 1, 15, 3)          0         
Total params: 3,507
Trainable params: 3,507
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.layers.re